In [ ]:
import os,sys
import torch
import numpy as np
import yaml
import chart_studio.plotly as py
import plotly
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=False)

SOFTWARE_DIR = '/home/twongjirad/working/larbys/lartpc_mlreco3d/'
sys.path.insert(0, SOFTWARE_DIR)
from mlreco.visualization import scatter_points, plotly_layout3d
from mlreco.visualization.gnn import scatter_clusters, network_topology, network_schematic
from mlreco.utils.ppn import uresnet_ppn_type_point_selector
from mlreco.utils.cluster.dense_cluster import fit_predict_np, gaussian_kernel
from mlreco.main_funcs import process_config, prepare
#from mlreco.utils.gnn.cluster import get_cluster_label
#from mlreco.utils.deghosting import adapt_labels_numpy as adapt_labels
from mlreco.visualization.gnn import network_topology

In [ ]:
# WE ARE USING LARCV1 (the submodule included in the ICDL repository)
import ROOT as rt
from larcv import larcv
from larlite import larlite
from larflow import larflow
from ROOT import larutil

In [ ]:
# SETUP GEOMETRY: FOR VISUALIZATION
DETECTOR = "uboone"

if DETECTOR == "icarus":
    detid = larlite.geo.kICARUS
elif DETECTOR == "uboone":
    detid = larlite.geo.kMicroBooNE
elif DETECTOR == "sbnd":
    detid = larlite.geo.kSBND
    
larutil.LArUtilConfig.SetDetector(detid)

import lardly
from lardly.detectoroutline import get_tpc_boundary_plot

tpclines = get_tpc_boundary_plot()

In [ ]:
# SETUP DATA LOADERS
datapath="/home/twongjirad/working/larbys/icarus/icdl/larflow/larmatchnet/dataprep/test.root"

cfg = """
iotool:
  batch_size: 1
  shuffle: False
  num_workers: 1
  collate_fn: CollateSparse
  dataset:
    name: LArCVDataset
    data_keys:
      - %s
    limit_num_files: 10
    #event_list: '[6436, 562, 3802, 6175, 15256]'
    schema:
      input_data:
        - parse_sparse3d
        - sparse3d_charge_plane0
        - sparse3d_charge_plane1
        - sparse3d_charge_plane2
      segment_label:
        - parse_sparse3d
        - sparse3d_semantics_ghost
      particles_label:
        - parse_particle_points
        - sparse3d_charge_plane2
        - particle_corrected
"""%(datapath)

In [ ]:
cfg=yaml.load(cfg,Loader=yaml.Loader)
# pre-process configuration (checks + certain non-specified default settings)
process_config(cfg)
# prepare function configures necessary "handlers"
hs=prepare(cfg)

In [ ]:
data = next(hs.data_io_iter)
data.keys()
# input_data: [N,C]
# C = (i,j,k,b,feat1,feat2,...)
print('input_data: ',data['input_data'].shape) 
print('segment_label: ',data['segment_label'].shape)
print('particles_label: ',data['particles_label'].shape)

In [ ]:
entry = 0
print("select shape: ",np.sum(data['input_data'][:, 0] == entry))
input_data = data['input_data'][ (data['input_data'][:, 0].astype(np.int32) == entry),:]
segment_label = data['segment_label'][data['segment_label'][:, 0] == entry, -1]
particles_label = data['particles_label'][data['particles_label'][:, 0] == entry]
print("input_data.shape: ",input_data.shape)
print("segment_label.shape: ",segment_label.shape)
print("particles_label.shape: ",particles_label.shape)

In [ ]:
trace = []

ADC_MAX = 100.0

# ADD ALL POINTS
# trace+= scatter_points(input_data,markersize=1,color=input_data[:, -2], cmin=0,cmax=ADC_MAX)
# trace[-1].name = 'input_data'
# trace[-1].marker.colorscale='viridis'

# ADD ALL NON-GHOST POINTS
trace+= scatter_points(input_data[segment_label < 5],markersize=1,color=input_data[segment_label < 5, -2], cmin=0,cmax=ADC_MAX)
trace[-1].name = 'input_data (true noghost)'
trace[-1].marker.colorscale='viridis'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)

In [ ]:
# SEMANTIC LABELS
trace = []

trace+= scatter_points(input_data,markersize=1,color=segment_label, cmin=0,cmax=5, colorscale=plotly.colors.qualitative.D3)
trace[-1].name = 'Semantic segmentation labels (including ghost points)'
trace[-1].marker.colorscale='viridis'

#trace+= scatter_points(input_data[segment_label < 5],markersize=1,color=segment_label[segment_label < 5], cmin=0,cmax=5, colorscale=plotly.colors.qualitative.D3)
#trace[-1].name = 'Semantic segmentation labels (w/o ghost points)'
#trace[-1].marker.colorscale='viridis'

fig = go.Figure(data=trace,layout=plotly_layout3d())
fig.update_layout(legend=dict(x=1.0, y=0.8))

iplot(fig)